In [1]:
#!/usr/bin/env python
"""A short and simple example experiment with restarts.

The code is fully functional but mainly emphasises on readability.
Hence it neither produces any progress messages (which can be very
annoying for long experiments) nor provides batch distribution,
as `example_experiment.py` does.

To apply the code to a different solver, `fmin` must be re-assigned or
re-defined accordingly. For example, using `cma.fmin` instead of
`scipy.optimize.fmin` can be done like::

    import cma
    def fmin(fun, x0):
        return cma.fmin(fun, x0, 2, {'verbose':-9})

"""
from __future__ import division, print_function
import cocoex, cocopp  # experimentation and post-processing modules
import scipy.optimize  # to define the solver to be benchmarked
from numpy.random import rand  # for randomised restarts
import os, webbrowser  # to show post-processed results in the browser
import pandas as pd
import sys, pickle, datetime

In [2]:
### input
output_folder = "TEST_static100"
nfe_base = 1e+5# increase to 10, 100, ...

# # fmin is re-defined to call our solver
# def fmin(fun, lbounds, ubounds, dim, budget):
#     result = solvers.franken10(50, fun, lbounds, ubounds, dim, 1000)
#     return result

In [3]:
### prepare
suite = cocoex.Suite("bbob", "", "function_indices:1,15 dimensions:10,20 instance_indices:1-10")
observer = cocoex.Observer("bbob", "result_folder: " + output_folder)
minimal_print = cocoex.utilities.MiniPrint()

In [4]:
def process_training_log():
    #load data
    data_log = pd.read_csv("./results/"+experiment_name+"/log.csv", header=None)

    #header
    header = ["GENERATIONS", "PRECISION", "FIT_MEDIAN", "FIT_ELITE"]
    for i in range(1, data_log.shape[1] - 3):
        header.append("FIT_INDV_"+str(i))
    data_log.columns = header
    
    return data_log

def get_best_indv():
    pdlog = process_training_log()
    filter_precision = pdlog[pdlog['PRECISION'] == pdlog['PRECISION'].min()]
    
    generation = -1
    fitness = 0
    for i, row in filter_precision.filter(regex="FIT_INDV").iterrows():
        bestfitness = row.max()
    
        if bestfitness >= fitness:
            generation = filter_precision['GENERATIONS'][i]
            fitness    = bestfitness
            
    print("Best at generation:", generation, "Fitness:", fitness, "(",experiment_name,")\n")
    return generation

In [5]:
experiment_name = "1K_static100"
#experiment_name = "190327_partial/190325_04"

#get best code
file = open("./results/"+experiment_name+"/"+str(get_best_indv())+".txt", 'r')
#file = open("./results/"+experiment_name+"/best.txt", 'r')
best = file.readlines()
file.close()

code = "import numpy as np\nfrom src.src.solution import Solution\nimport src.src.operators as op\n"
for line in best[6:-8]:
    code += line
    
print(code)

Best at generation: 100 Fitness: 1.0 ( 1K_static100 )

import numpy as np
from src.src.solution import Solution
import src.src.operators as op
def ge(n, my_func, bounds, dimension, max_nfe):
  Solution.setProblem(my_func, bounds, dimension, maximize=False)
  Solution.repair = op.repair_random
  X = Solution.initialize(n)
  for Xi in X:    Xi.setX(op.init_random(*Solution.bounds, Solution.dimension))
  [Xi.getFitness() for Xi in X]
  Solution.updateHistory(X)
  while Solution.nfe < max_nfe:
    U = X
    #Round 1
    S1 = op.select_tournament(U, n=1, k=int(n*0.10))
    U = op.w_levy_flight(S1)
    X  = U
    #Round 2
    S1 = op.select_current(U)
    S2 = op.select_random(X, 1)
    S3 = op.select_current(X)
    U  = op.w_mut_de(S1, S2, S3, beta=0.50)
    X  = U
    [Xi.getFitness() for Xi in X]
  return X
ge(100, my_func, bounds, dimension, max_nfe)
XXX_output_XXX = Solution.best.getFitness()



In [ ]:
### go
start = datetime.datetime.now()
print(start)
for problem in suite:  # this loop will take several minutes or longer
    problem.observe_with(observer)  # generates the data for cocopp post-processing
    # apply restarts while neither the problem is solved nor the budget is exhausted
    max_nfe = nfe_base*problem.dimension
    
    d = {
        "max_nfe"  : max_nfe, 
        "dimension": problem.dimension,
        "my_func"  : problem,
        "bounds"   : (problem.lower_bounds[0], problem.upper_bounds[0])
        }
    exec(code, d)
    minimal_print(problem, final=problem.index == len(suite) - 1)
    print(problem.id, " finished at ",datetime.datetime.now())
time_lapse = datetime.datetime.now() - start


2019-03-28 13:22:09.017642
10D: .bbob_f001_i01_d10  finished at  2019-03-28 13:24:36.037871
.bbob_f001_i02_d10  finished at  2019-03-28 13:27:00.943797
.bbob_f001_i03_d10  finished at  2019-03-28 13:29:15.392503
.bbob_f001_i04_d10  finished at  2019-03-28 14:12:33.736512
.bbob_f001_i05_d10  finished at  2019-03-28 14:34:18.856686
.bbob_f001_i71_d10  finished at  2019-03-28 14:36:45.852287
.bbob_f001_i72_d10  finished at  2019-03-28 14:39:33.950728
.bbob_f001_i73_d10  finished at  2019-03-28 14:42:21.209245
.bbob_f001_i74_d10  finished at  2019-03-28 14:45:05.191080
.bbob_f001_i75_d10  finished at  2019-03-28 14:47:33.032684
.bbob_f015_i01_d10  finished at  2019-03-28 14:50:01.569968
.bbob_f015_i02_d10  finished at  2019-03-28 14:52:18.535528
.bbob_f015_i03_d10  finished at  2019-03-28 14:54:37.275738
.bbob_f015_i04_d10  finished at  2019-03-28 14:57:06.035031


In [18]:
### post-process data
cocopp.main(observer.result_folder)  # re-run folders look like "...-001" etc
webbrowser.open("file://" + os.getcwd() + "/ppdata/index.html")

Post-processing (1)
  Using:
    ./exdata/1K_Static1

Post-processing (1)
  Will generate output data in folder ppdata/1K_Static1
    this might take several minutes.
Scaling figures...
Loading best algorithm data from refalgs/best2009-bbob.tar.gz ...


/anaconda3/lib/python3.7/site-packages/cocopp-2.2.1.10-py3.7.egg/cocopp/pproc.py:1103: UserWarning:   less than 15 instances in [1, 2, 3, 4, 5, 71, 72, 73, 74, 75] (f1, 10D)
  str(self.dim) + 'D)')


  Data consistent according to consistency_check() in pproc.DataSet
  using: /anaconda3/lib/python3.7/site-packages/cocopp-2.2.1.10-py3.7.egg/cocopp/refalgs/best2009-bbob.tar.gz
  done (Wed Mar 27 15:44:30 2019).
  done (Wed Mar 27 15:44:31 2019).
Generating LaTeX tables...
  done (Wed Mar 27 15:44:31 2019).
ECDF graphs...


/anaconda3/lib/python3.7/site-packages/cocopp-2.2.1.10-py3.7.egg/cocopp/testbedsettings.py:175: ResourceWarning: unclosed file <_io.TextIOWrapper name='/anaconda3/lib/python3.7/site-packages/cocopp-2.2.1.10-py3.7.egg/cocopp/bbob-benchmarkinfos.txt' mode='r' encoding='UTF-8'>
  self.info_filename)).readlines():


  done (Wed Mar 27 15:44:35 2019).
ECDF graphs per function...
  done (Wed Mar 27 15:44:41 2019).
aRT loss ratio figures and tables...
  done (Wed Mar 27 15:44:43 2019).
Output data written to folder /Users/annabogdanova/Documents/Codes/SwarmGE/GECCO19/ppdata/1K_Static1
Setting changes in `cocopp.genericsettings` compared to default:
    simulated_runlength_bootstrap_sample_size: from 10 to 10.098990100989901
ALL done (Wed Mar 27 15:44:43 2019).


True

In [27]:
# Var 1: 22 functions, 5 inst, 20D, 40D, e+4
(6 * 2 * 22)/60

4.4

In [ ]:
#